### Ref 
- https://github.com/llSourcell/recommender_live/blob/master/Song%20Recommender_Python.ipynb

- https://towardsdatascience.com/how-to-build-a-simple-song-recommender-296fcbc8c85

In [44]:
%matplotlib inline

import pandas as pd 
from sklearn.cross_validation import train_test_split
import numpy as np
import time
from sklearn.externals import joblib

In [45]:
cd ..

/Users/yennanliu


In [88]:
def get_data():
    route='datasets/ml-latest-small/'
    #df_movie = pd.read_csv(route +'movies.csv')
    df_ratings = pd.read_csv(route +'ratings.csv')
    return df_ratings

def data_preprocess(df):
    df['view_count']=df.groupby(['userId','movieId']).count().reset_index()['rating']
    movie_grouped = df.groupby(['movieId']).agg({'view_count': 'count'}).reset_index()
    grouped_sum = movie_grouped['view_count'].sum()
    movie_grouped['percentage']  = movie_grouped['view_count'].div(grouped_sum)*100
    movie_grouped.sort_values(['view_count', 'movieId'], ascending = [0,1])
    return movie_grouped



def get_train_test_data(df):
	train_data, test_data = train_test_split(df, test_size = 0.20, random_state=0)
	return train_data, test_data



# ML

class popularity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.popularity_recommendations = None
        
    #Create the popularity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

        #Get a count of user_ids for each unique song as recommendation score
        train_data_grouped = train_data.groupby([self.item_id]).agg({self.user_id: 'count'}).reset_index()
        train_data_grouped.rename(columns = {'userId': 'score'},inplace=True)
    
        #Sort the songs based upon recommendation score
        train_data_sort = train_data_grouped.sort_values(['score', self.item_id], ascending = [0,1])
    
        #Generate a recommendation rank based upon score
        train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
        
        #Get the top 10 recommendations
        self.popularity_recommendations = train_data_sort.head(10)

    #Use the popularity based recommender system model to
    #make recommendations
    def recommend(self, user_id):    
        user_recommendations = self.popularity_recommendations
        print ('---------')
        print (user_recommendations)
        
        #Add user_id column for which the recommendations are being generated
        user_recommendations['userId'] = user_id
        print ('---------')
        print (user_recommendations)
    
        #Bring user_id column to the front
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols]
        
        return user_recommendations



In [98]:
df_ratings.userId.unique()[1]

2

In [79]:
#len(df_ratings)
df_ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [48]:
df_ratings['count']=df_ratings.groupby(['userId','movieId']).count().reset_index()['rating']




In [49]:
#df_ratings.groupby(['userId','movieId']).count()

In [52]:
cd movie_recommendation/

/Users/yennanliu/movie_recommendation


In [55]:
df_ratings = get_data()
movie_grouped = data_preprocess(df_ratings)
print (movie_grouped)
train_data, test_data = get_train_test_data(df_ratings)



      movieId  view_count  percentage
0           1         247    0.246990
1           2         107    0.106996
2           3          59    0.058998
3           4          13    0.012999
4           5          56    0.055998
5           6         104    0.103996
6           7          53    0.052998
7           8           5    0.005000
8           9          20    0.019999
9          10         122    0.121995
10         11          82    0.081997
11         12          18    0.017999
12         13           8    0.008000
13         14          31    0.030999
14         15          11    0.011000
15         16          88    0.087996
16         17          86    0.085997
17         18          26    0.025999
18         19          92    0.091996
19         20          13    0.012999
20         21          95    0.094996
21         22          38    0.037998
22         23          22    0.021999
23         24          34    0.033999
24         25         101    0.100996
25         2

In [91]:
### run ML ###

pm = popularity_recommender_py()
pm.create(train_data, 'userId', 'movieId')

In [56]:
df_ratings.head(3)

,userId,movieId,rating,timestamp,view_count
0,1,31,2.5,1260759144,1
1,1,1029,3.0,1260759179,1
2,1,1061,3.0,1260759182,1


In [92]:
user_id = users[5]
pm.recommend(user_id)

---------
      movieId  score  Rank
264       296    267   1.0
318       356    267   2.0
282       318    249   3.0
519       593    233   4.0
230       260    230   5.0
423       480    215   6.0
2005     2571    213   7.0
0           1    209   8.0
1001     1270    194   9.0
529       608    192  10.0
---------
      movieId  score  Rank  userId
264       296    267   1.0       6
318       356    267   2.0       6
282       318    249   3.0       6
519       593    233   4.0       6
230       260    230   5.0       6
423       480    215   6.0       6
2005     2571    213   7.0       6
0           1    209   8.0       6
1001     1270    194   9.0       6
529       608    192  10.0       6


,userId,movieId,score,Rank
264,6,296,267,1.0
318,6,356,267,2.0
282,6,318,249,3.0
519,6,593,233,4.0
230,6,260,230,5.0
423,6,480,215,6.0
2005,6,2571,213,7.0
0,6,1,209,8.0
1001,6,1270,194,9.0
529,6,608,192,10.0


In [93]:
user_id = users[199]
pm.recommend(user_id)

---------
      movieId  score  Rank  userId
264       296    267   1.0       6
318       356    267   2.0       6
282       318    249   3.0       6
519       593    233   4.0       6
230       260    230   5.0       6
423       480    215   6.0       6
2005     2571    213   7.0       6
0           1    209   8.0       6
1001     1270    194   9.0       6
529       608    192  10.0       6
---------
      movieId  score  Rank  userId
264       296    267   1.0     200
318       356    267   2.0     200
282       318    249   3.0     200
519       593    233   4.0     200
230       260    230   5.0     200
423       480    215   6.0     200
2005     2571    213   7.0     200
0           1    209   8.0     200
1001     1270    194   9.0     200
529       608    192  10.0     200


,userId,movieId,score,Rank
264,200,296,267,1.0
318,200,356,267,2.0
282,200,318,249,3.0
519,200,593,233,4.0
230,200,260,230,5.0
423,200,480,215,6.0
2005,200,2571,213,7.0
0,200,1,209,8.0
1001,200,1270,194,9.0
529,200,608,192,10.0


In [58]:
users = df_ratings['userId'].unique()
print (len(users)) ## return 671 unique users
movies = df_ratings['movieId'].unique()
print (len(movies))  ## return 9066 unique songs

671
9066


In [59]:
df_ratings.head(3)

,userId,movieId,rating,timestamp,view_count
0,1,31,2.5,1260759144,1
1,1,1029,3.0,1260759179,1
2,1,1061,3.0,1260759182,1


In [60]:
pm = Recommenders.popularity_recommender_py()
pm.create(train_data, 'user_id', 'song')
#user the popularity model to make some prediction
user_id = users[5]
pm.recommend(user_id)

NameError: name 'Recommenders' is not defined

In [61]:
movie_grouped

,movieId,view_count,percentage
0,1,247,0.246990
1,2,107,0.106996
2,3,59,0.058998
3,4,13,0.012999
4,5,56,0.055998
5,6,104,0.103996
6,7,53,0.052998
7,8,5,0.005000
8,9,20,0.019999
9,10,122,0.121995


In [62]:
df_ratings.head(3)

,userId,movieId,rating,timestamp,view_count
0,1,31,2.5,1260759144,1
1,1,1029,3.0,1260759179,1
2,1,1061,3.0,1260759182,1


In [63]:
cd movie_recommendation/

[Errno 2] No such file or directory: 'movie_recommendation/'
/Users/yennanliu/movie_recommendation


In [64]:
ls datasets

ml-latest/       ml-latest-small/


In [65]:
ls #datasets/ml-latest-small

README.md                         movie_recommend_popularity.py
archive/                          movie_recommend_spark_CF.ipynb
datasets/                         movie_recommend_spark_CF.py
download_dataset.sh               movie_recommend_spark_copy.py
images/                           notebook/
init.sh                           ref/
install_pyspark.sh                test.py
launch_env.sh                     test.sh
movie_recommend_popularity.ipynb  utility_data_load.py


In [66]:
cd datasets/ml-latest-small

/Users/yennanliu/movie_recommendation/datasets/ml-latest-small


In [67]:
df_movie = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')
df_tags = pd.read_csv('tags.csv')
df_links = pd.read_csv('links.csv')

In [68]:
df_movie.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [69]:
df_ratings.head(1)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [70]:
df_tags.head(1)

,userId,movieId,tag,timestamp
0,15,339,sandra 'boring' bullock,1138537770


In [71]:
df_links.head(1)

,movieId,imdbId,tmdbId
0,1,114709,862.0
